In [62]:
%load_ext lab_black
import argparse
import contextlib
import datetime
import io
import logging
import multiprocessing
import os
import random
import sys
from itertools import chain, combinations
from timeit import default_timer as timer

import altair as alt
import altair_viewer
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import peewee
from evolutionary_search import EvolutionaryAlgorithmSearchCV
from json_tricks import dumps, loads
from playhouse.shortcuts import model_to_dict
from scipy.stats import randint, uniform
from sklearn.datasets import load_iris
from tabulate import tabulate
from IPython.core.display import display, HTML

from cluster_strategies import (
    DummyClusterStrategy,
    MostUncertainClusterStrategy,
    RandomClusterStrategy,
    RoundRobinClusterStrategy,
)
from dataStorage import DataStorage
from experiment_setup_lib import (
    ExperimentResult,
    classification_report_and_confusion_matrix,
    get_db,
    get_single_al_run_stats_row,
    get_single_al_run_stats_table_header,
    load_and_prepare_X_and_Y,
)
from sampling_strategies import (
    BoundaryPairSampler,
    CommitteeSampler,
    RandomSampler,
    UncertaintySampler,
)

alt.renderers.enable("altair_viewer")
#  alt.renderers.enable('vegascope')

config = {
    "datasets_path": "../datasets",
    "db": "tunnel",
    "param_list_id": "best_global_score",
}

db = get_db(db_name_or_type=config["db"])

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [63]:
# select count(*), dataset_name from experimentresult group by dataset_name;
results = ExperimentResult.select(
    ExperimentResult.dataset_name,
    peewee.fn.COUNT(ExperimentResult.id_field).alias("dataset_name_count"),
).group_by(ExperimentResult.dataset_name)

for result in results:
    print("{:>4,d} {}".format(result.dataset_name_count, result.dataset_name))

1,427 orange
1,410 sylva
1,428 hiva
1,436 ibn_sina
1,438 dwtc
1,398 zebra


In [102]:
#  SELECT param_list_id, avg(fit_score), stddev(fit_score), avg(global_score), stddev(global_score), avg(start_set_size) as sss, count(*) FROM experimentresult WHERE start_set_size = 1 GROUP BY param_list_id ORDER BY 7 DESC, 4 DESC LIMIT 30;

results = (
    ExperimentResult.select(
        ExperimentResult.param_list_id,
        ExperimentResult.fit_score,
        ExperimentResult.global_score_no_weak_acc,
        ExperimentResult.amount_of_user_asked_queries,
        ExperimentResult.classifier,
        ExperimentResult.global_score_no_weak_acc,
        ExperimentResult.amount_of_user_asked_queries,
        ExperimentResult.dataset_name,
        ExperimentResult.test_fraction,
        ExperimentResult.sampling,
        ExperimentResult.cluster,
        ExperimentResult.nr_queries_per_iteration,
        ExperimentResult.with_uncertainty_recommendation,
        ExperimentResult.with_cluster_recommendation,
        ExperimentResult.uncertainty_recommendation_certainty_threshold,
        ExperimentResult.uncertainty_recommendation_ratio,
        ExperimentResult.cluster_recommendation_minimum_cluster_unity_size,
        ExperimentResult.cluster_recommendation_ratio_labeled_unlabeled,
        ExperimentResult.allow_recommendations_after_stop,
        ExperimentResult.stopping_criteria_uncertainty,
        ExperimentResult.stopping_criteria_acc,
        ExperimentResult.stopping_criteria_std,
        ExperimentResult.experiment_run_date,
    )
    .where(
        (ExperimentResult.amount_of_user_asked_queries < 10000)
        & (ExperimentResult.dataset_name == "dwtc")
    )
    .order_by(
        # ExperimentResult.id_field.desc(),
        ExperimentResult.global_score_no_weak_acc.desc(),
    )
    .limit(20)
)

table = []
id = 0
for result in results:
    data = {**{"id": id}, **vars(result)["__data__"]}
    # data["param_list_id"] = data["__data__"]["param_list_id"]
    # del data["__data__"]
    # del data["_dirty"]
    # del data["__rel__"]

    # data = {**data, **vars(one_param_list_id_result)["__data__"]}

    table.append(data)
    id += 1

display(HTML(tabulate(table, headers="keys", tablefmt="html")))

id,param_list_id,fit_score,global_score_no_weak_acc,amount_of_user_asked_queries,classifier,dataset_name,test_fraction,sampling,cluster,nr_queries_per_iteration,with_uncertainty_recommendation,with_cluster_recommendation,uncertainty_recommendation_certainty_threshold,uncertainty_recommendation_ratio,cluster_recommendation_minimum_cluster_unity_size,cluster_recommendation_ratio_labeled_unlabeled,allow_recommendations_after_stop,stopping_criteria_uncertainty,stopping_criteria_acc,stopping_criteria_std,experiment_run_date
0,b0c58e7a27384ff8e903ab0722b65ffc,0.250387,0.82138,2464,RF,dwtc,0.5,uncertainty_max_margin,dummy,10,False,True,0.987832,0.001,0.960142,0.897887,False,0.940698,0.799782,0.396984,2020-03-21 17:29:26.003928
1,1614085c521cb0b320077d2549f10528,0.313187,0.821081,2334,RF,dwtc,0.5,uncertainty_max_margin,dummy,10,False,True,0.815775,0.0001,0.996774,0.653752,False,0.691711,0.883871,0.229771,2020-03-19 13:43:04.061168
2,8db937326c36f62aba22917f6b212e21,0.299374,0.819557,2364,RF,dwtc,0.5,uncertainty_max_margin,dummy,10,False,True,0.900506,0.001,0.866525,0.702983,True,0.974607,0.652401,0.250175,2020-03-21 08:38:26.684211
3,6b5678ec00e2f38d1944d9c0d9a5ca50,0.415615,0.818991,2094,RF,dwtc,0.5,uncertainty_max_margin,dummy,10,False,True,0.576296,0.001,0.945271,0.958353,True,0.3143,0.384919,0.669097,2020-03-21 09:55:57.160653
4,eb70b737e7f02d0d38926dfe9e018cd7,0.391867,0.818171,2154,RF,dwtc,0.5,uncertainty_max_margin,dummy,10,False,True,0.650672,0.1,0.789006,0.758648,True,0.0600979,0.138718,0.626503,2020-03-18 20:02:49.104861
5,cfec75eed9d2347b5d45aa4fae7d456b,0.404177,0.817435,2124,RF,dwtc,0.5,uncertainty_max_margin,dummy,10,False,True,0.958689,0.1,0.751011,0.797977,False,0.0649347,0.469157,0.639028,2020-03-14 11:35:47.276258
6,7930743fd316cc11490934e68886cab1,0.27519,0.815844,2414,RF,dwtc,0.5,uncertainty_max_margin,dummy,10,False,True,0.863349,0.01,0.536573,0.95402,True,0.0449067,0.779072,0.431601,2020-03-17 18:46:16.780501
7,54ac87761baa9944f1661fc3a31fe1ac,0.176157,0.815244,2604,RF,dwtc,0.5,uncertainty_max_margin,dummy,10,False,True,0.938347,0.001,0.644337,0.956295,True,0.505081,0.434012,0.279138,2020-03-13 16:14:49.213377
8,5606d5e9066fd35feb9a740390d200a6,0.198095,0.814917,2564,RF,dwtc,0.5,uncertainty_max_margin,dummy,10,False,True,0.948614,0.1,0.874997,0.831778,False,0.574678,0.39799,0.139449,2020-03-18 04:39:59.621756
9,32878448f07b5ac2f1becfef44a79c52,0.0392366,0.814505,2830,RF,dwtc,0.5,uncertainty_lc,MostUncertain_lc,10,False,True,0.5257,0.001,0.575902,0.946689,False,0.296145,0.294903,0.121271,2020-03-15 02:51:18.404996


In [103]:
# SELECT id_field, param_list_id, dataset_path, start_set_size as sss, sampling, cluster, allow_recommendations_after_stop as SA, stopping_criteria_uncertainty as SCU, stopping_criteria_std as SCS, stopping_criteria_acc as SCA, amount_of_user_asked_queries as "#q", acc_test, fit_score, global_score_norm, thread_id, end_time from experimentresult where param_list_id='31858014d685a3f1ba3e4e32690ddfc3' order by end_time, fit_score desc, param_list_id;
loaded_data = {}


def pre_fetch_data(top_n=0):
    best_param_list_id = table[top_n]["param_list_id"]

    results = (
        ExperimentResult.select()
        .where(ExperimentResult.param_list_id == best_param_list_id)
        .order_by(ExperimentResult.dataset_name)
    )

    loaded_data[top_n] = []
    for result in results:
        loaded_data[top_n].append(result)
    print("Loaded Top " + str(top_n) + " data")


pre_fetch_data(0)

Loaded Top 0 data


In [104]:
def visualise_top_n(top_n=0):
    charts = []

    alt.renderers.enable("html")

    for result in loaded_data[top_n][:]:
        metrics = loads(result.metrics_per_al_cycle)
        test_data_metrics = [
            metrics["test_data_metrics"][0][f][0]["weighted avg"]
            for f in range(0, len(metrics["test_data_metrics"][0]))
        ]
        test_acc = [
            metrics["test_data_metrics"][0][f][0]["accuracy"]
            for f in range(0, len(metrics["test_data_metrics"][0]))
        ]

        data = pd.DataFrame(
            {
                "iteration": range(0, len(metrics["all_unlabeled_roc_auc_scores"])),
                "all_unlabeled_roc_auc_scores": metrics["all_unlabeled_roc_auc_scores"],
                "query_length": metrics["query_length"],
                "recommendation": metrics["recommendation"],
                "query_strong_accuracy_list": metrics["query_strong_accuracy_list"],
                "f1": [i["f1-score"] for i in test_data_metrics],
                "test_acc": test_acc,
                #'asked_queries': [sum(metrics['query_length'][:i]) for i in range(0, len(metrics['query_length']))],
            }
        )

        # bar width
        data["asked_queries"] = data["query_length"].cumsum()
        data["asked_queries_end"] = data["asked_queries"].shift(fill_value=0)

        # print(data[['asked_queries', 'query_length']])

        data["recommendation"] = data["recommendation"].replace(
            {
                "A": "Oracle",
                "C": "Weak Cluster",
                "U": "Weak Certainty",
                "G": "Ground Truth",
            }
        )

        # data = data[:100]

        # calculate global score OHNE

        chart = (
            alt.Chart(data)
            .mark_rect(
                # point=True,
                # line=True,
                # interpolate='step-after',
            )
            .encode(
                x=alt.X("asked_queries_end", title="#asked queries (weak and oracle)"),
                x2="asked_queries",
                color=alt.Color("recommendation", scale=alt.Scale(scheme="tableau10")),
                tooltip=[
                    "iteration",
                    "f1",
                    "test_acc",
                    "all_unlabeled_roc_auc_scores",
                    "query_strong_accuracy_list",
                    "query_length",
                    "recommendation",
                ],
                # scale=alt.Scale(domain=[0,1])
            )
            .properties(title=result.dataset_name)
            .interactive()
        )
        charts.append(
            alt.hconcat(
                chart.encode(
                    alt.Y(
                        "all_unlabeled_roc_auc_scores", scale=alt.Scale(domain=[0, 1])
                    )
                ).properties(title=result.dataset_name + ": roc_auc"),
                # chart.encode(alt.Y('f1', scale=alt.Scale(domain=[0,1]))).properties(title=result.dataset_name + ': f1'),
                chart.encode(
                    alt.Y("test_acc", scale=alt.Scale(domain=[0, 1]))
                ).properties(title=result.dataset_name + ": test_acc"),
            )
        )

    return alt.vconcat(*charts).configure()


visualise_top_n(0)

alt.VConcatChart(...)

In [105]:
pre_fetch_data(1)
visualise_top_n(1)

Loaded Top 1 data


alt.VConcatChart(...)

In [94]:
pre_fetch_data(2)
visualise_top_n(2)

Loaded Top 2 data


alt.VConcatChart(...)

In [95]:
pre_fetch_data(3)
visualise_top_n(3)

Loaded Top 3 data


alt.VConcatChart(...)

In [96]:
pre_fetch_data(4)
visualise_top_n(4)

Loaded Top 4 data


alt.VConcatChart(...)

In [97]:
pre_fetch_data(5)
visualise_top_n(5)

Loaded Top 5 data


alt.VConcatChart(...)